#### stores data of landmarks for all data into .csv under pose_data folder

In [20]:
import os
import cv2
import json
import pandas as pd
import mediapipe as mp

In [21]:
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

custom_style = mp_drawing_styles.get_default_pose_landmarks_style()
custom_connections = list(mp_pose.POSE_CONNECTIONS)

In [22]:
def detect(img):

    data = []
    # Create a Pose object
    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        # Process the image to detect body pose
        results = pose.process(img)

        if results.pose_landmarks:
            # Draw the landmarks on the image
            mp_drawing.draw_landmarks(
                img, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
        
            # Save the landmarks in a DataFrame
            for landmark in results.pose_landmarks.landmark:
                data += [landmark.x, landmark.y, landmark.z, landmark.visibility]

    return img, data

In [23]:
if not os.path.exists('pose_data'):
    os.makedirs('pose_data')

poses = os.listdir('data')
poses.remove('.DS_Store')

for i in poses:
    files = os.listdir('data/' + i)

    pose_data = []
    for j in files:
        img = cv2.imread('data/' + i + '/' + j)

        if img is None:
            continue
        
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img, img_data = detect(img)
        if img_data != []:
            pose_data.append(img_data)

        print('Processed: ' + j)

    df = pd.DataFrame(pose_data)
    df.to_csv( 'pose_data/' + i + '_data.csv', index=False)

libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
Premature end of JPEG file
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
Corrupt JPEG data: premature end of data segment
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile


In [ ]:
# making a mapping json file
mapping = {}
poses.sort()

for i in range(len(poses)):
    mapping[i] = poses[i]

with open('mapping.json', 'w') as f:
    json.dump(mapping, f, indent=4)